In [1]:
from CollabFilterOneVectorPerItem import CollabFilterOneVectorPerItem
from train_valid_test_loader import load_train_valid_test_datasets
import matplotlib.pyplot as plt
import numpy as np

# Load the dataset
train_tuple, valid_tuple, test_tuple, n_users, n_items = \
    load_train_valid_test_datasets()

In [2]:
alpha = 0.0
K_values = [2, 10, 50]

models_by_K = {}

for K in K_values:
    print(f"==================K={K}===========")
    model = CollabFilterOneVectorPerItem(
        n_epochs=10, batch_size=10000, step_size=0.1,
        n_factors=K, alpha=alpha)
    model.init_parameter_dict(n_users, n_items, train_tuple)
    model.fit(train_tuple, valid_tuple)
    
    # print(f"model.trace_loss: {model.trace_loss}")
    # print(f"model.trace_epoch: {model.trace_epoch}")
    # print(f"model.trace_auc_train: {model.trace_auc_train}")
    # print(f"model.trace_auc_valid: {model.trace_auc_valid}")
    
    models_by_K[K] = model

==================K=2===========
epoch       0.000 | loss_total     1.55557 | train_AUC    0.50000 | valid_AUC     0.50000 | grad_wrt_mu     1.05535 | grad_wrt_b_per_user     0.00136 | grad_wrt_c_per_item     0.00082 | grad_wrt_U     0.02605 | grad_wrt_V     0.01543
epoch       0.143 | loss_total     1.39900 | train_AUC    0.70033 | valid_AUC     0.69044 | grad_wrt_mu     0.70252 | grad_wrt_b_per_user     0.00111 | grad_wrt_c_per_item     0.00072 | grad_wrt_U     0.01904 | grad_wrt_V     0.01347
epoch       0.286 | loss_total     1.29691 | train_AUC    0.71205 | valid_AUC     0.70142 | grad_wrt_mu     0.41200 | grad_wrt_b_per_user     0.00094 | grad_wrt_c_per_item     0.00064 | grad_wrt_U     0.01606 | grad_wrt_V     0.00995
epoch       0.429 | loss_total     1.30394 | train_AUC    0.71985 | valid_AUC     0.70656 | grad_wrt_mu     0.34279 | grad_wrt_b_per_user     0.00090 | grad_wrt_c_per_item     0.00057 | grad_wrt_U     0.01762 | grad_wrt_V     0.01126
epoch       0.571 | loss_total 

/home/thayes/miniconda3/envs/cs135_env/lib/python3.10/site-packages/autograd/tracer.py:48: RuntimeWarning: overflow encountered in multiply
  return f_raw(*args, **kwargs)
/home/thayes/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:88: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/projects/school/CS-135/projectB/CollabFilterOneVectorPerItem.py:104: RuntimeWarning: overflow encountered in multiply
  factor_mult = user_factor_items * item_factor_items


ValueError: Input contains NaN.

In [ ]:
fig, ax = plt.subplots(1, 3, sharex="row")

for i, K in enumerate(K_values):
    model = models_by_K[K]
    # user_id_N, item_id_N, y_N = test_tuple
    # yhat_N = model.predict(user_id_N, item_id_N)
    mae_by_epoch = model.trace_mae_valid
    print("mae:", mae_by_epoch)
    a = ax[i]
    a.set_title(f"K = {K}")
    a.plot(mae_by_epoch)